In [1]:
## Model processing notebook


In [2]:
import pandas as pd
import json
from os import listdir
import os
import re
import pymongo
import pickle

## Steps here:

1. Import data from pickle file
1. Run text processing function to get feature matrix
1. Run cross-validation on a classifier




------------------------------
## Step 1

In [3]:
# Step 1
filepath = 'FlaskApp/data/full_data.pkl'

data = pd.read_pickle(filepath)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13316 entries, 0 to 13315
Data columns (total 3 columns):
FROM    13316 non-null object
TO      13316 non-null object
TEXT    13316 non-null object
dtypes: object(3)
memory usage: 416.1+ KB


In [8]:
data[['TO','TEXT']].groupby('TO').count()

,TEXT
TO,
CEO,457
Director,847
Employee,4165
In House Lawyer,56
Manager,1087
Manager),215
Managing Director,413
President,1137
Trader,681


------------------------------
## Step 2

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

class Processor(object):
    
    def __init__(self):
        self.myVect = None
        pass
    
    def fit_transform(self, X, flag=True):
        
        
        
        if flag:
            self.myVect = TfidfVectorizer()
            self.myVect.fit(X)
            
        output = self.myVect.transform(X)
        
        return output.toarray()
    
    def fit(self, *args):
        self.fit_transform(*args)
    
    def transform(self, X):
        output = self.fit_transform(X, flag=False)
        return output
  

------------------------------
## Step 3

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score

#testing purposes
myData = data


myFold = KFold(len(myData.TO), n_folds=5, shuffle=True)

accuracyScores = []

for trainIndex, testIndex in myFold:
    myModel = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    myProcessor = Processor()
    
    trainX = myProcessor.fit_transform(data.TEXT[trainIndex])
    trainY = myData.FROM[trainIndex]
    
    testX = myProcessor.transform(myData.TEXT[testIndex])
    testY = myData.FROM[testIndex]
    
    myModel.fit(trainX, trainY)
    predicts = myModel.predict(testX)
    
    accuracy = accuracy_score(testY, predicts)
    accuracyScores.append(accuracy)
    print 'Accuracy score: ', accuracy
    
print 'Overall Model accuracy: ', np.mean(accuracy)

# Summary Code

#Train Full Model
myModel = RandomForestClassifier(n_estimators=100, n_jobs=-1)
myProcessor = Processor()

trainX = myProcessor.fit_transform(data.TEXT)
trainY = myData.FROM

testX = myProcessor.transform(myData.TEXT)
testY = myData.FROM

myModel.fit(trainX, trainY)
predicts = myModel.predict(testX)

accuracy = accuracy_score(testY, predicts)

print 'Full set Accuracy score: ', accuracy

Accuracy score:  0.907657657658
Accuracy score:  0.90161472024
Accuracy score:  0.90311678558
Accuracy score:  0.903492301915
Accuracy score:  0.891100262861
Overall Model accuracy:  0.891100262861
Full set Accuracy score:  0.9980474617


0.57849445989930004

In [6]:
minidata = data[:5]

In [26]:
miniFold = KFold(len(minidata.FROM), n_folds=5)

In [32]:
for trainIndex, testIndex in miniFold:
    print trainIndex, testIndex
    print minidata.FROM[trainIndex]

[1 2 3 4] [0]
1    Manager
2    Manager
3    Manager
4    Manager
Name: FROM, dtype: object
[0 2 3 4] [1]
0    Manager
2    Manager
3    Manager
4    Manager
Name: FROM, dtype: object
[0 1 3 4] [2]
0    Manager
1    Manager
3    Manager
4    Manager
Name: FROM, dtype: object
[0 1 2 4] [3]
0    Manager
1    Manager
2    Manager
4    Manager
Name: FROM, dtype: object
[0 1 2 3] [4]
0    Manager
1    Manager
2    Manager
3    Manager
Name: FROM, dtype: object
